Setup Notes:
1. Register the model/model.joblib in your workspace under the name sklearn-iris
2. Create the necessary resources using the src/IaC/IaC_0_ray_serve_on_aml.azcli script
3. Get the external IP address of your many-models raycluster load balancer service and update the 'base_uri' variable accordingly below
   kubectl get service many-models-loadbalancer
4. If you update your code (many_models_serving.py), then you have to zip the folder script and push to ray repo. Update the line 15 in ray_service.yaml to point the working_dir to the zip file in your git repo
5. Execute the code below from a kernel based on src/tests/test_conda.yml conda environment

In [9]:
import json
import urllib.request
import random
import pandas as pd

base_uri ='http://20.118.113.102:8000'

dataset = pd.read_csv("https://azuremlexamples.blob.core.windows.net/datasets/iris.csv")
X=dataset[['sepal_length','sepal_width','petal_length','petal_width']]

def score(X, tenant):
    # tenant = random.choices(["tenant1","tenant2", "tenant3","tenant4","tenant5","tenant6","tenant7", "tenant8","tenant9"])[0]
    len = random.randint(2,50)
    data = {"tenant":tenant, "data": X.head(len).to_numpy().tolist()}
    body = str.encode(json.dumps(data))
    url = f'{base_uri}/score'
    headers = {'Content-Type':'application/json'}
    req = urllib.request.Request(url, body, headers)

    try:
        response = urllib.request.urlopen(req)
        result = response.read()
        print(json.loads(result))
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))
        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))

def updated_dedicated_pool(mapping):

    body = str.encode(json.dumps({"mapping":mapping}))
    url = f'{base_uri}/update_dedicated_pool'
    headers = {'Content-Type':'application/json'}
    req = urllib.request.Request(url, body, headers)
    try:
        response = urllib.request.urlopen(req)
        result = response.read()
        print(json.loads(result))
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))
        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))


In [10]:
score(X, "tenant25")

KeyboardInterrupt: 

API to update dedicated pool mapping. Try this to assign a tenant to a dedicated deployment

In [37]:
mapping ={"tenant19":"deployment9","tenant10":"deployment10"}
updated_dedicated_pool(mapping)

{'tenant19': 'deployment9', 'tenant10': 'deployment10'}
